In [16]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS from flask_cors
# import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
import pymysql
import numpy as np

app = Flask(__name__)
CORS(app) 

# # MySQL configuration
# db = pymysql.connect(
#     host='109.106.254.1',
#     user='u690159757_kim',
#     password='*yKa&T6y4#sJ8XvE9j',
#     db='u690159757_kim',
#     cursorclass=pymysql.cursors.DictCursor  
# )
# MySQL configuration
db = pymysql.connect(
    host='mysql5049.site4now.net',
    user='aa0682_movies',
    password='Password1234.',
    db='db_aa0682_movies',
    connect_timeout=8800,
    cursorclass=pymysql.cursors.DictCursor
     
)

In [17]:
cursor = db.cursor()
cursor.execute(''' SELECT * FROM watchedMovies''')
data = cursor.fetchall()[::-1]
db.commit()
cursor.close()

movies = [{'movie_id':row['movie_id']} for row in data]


In [18]:
movies

[{'movie_id': 30},
 {'movie_id': 412},
 {'movie_id': 7},
 {'movie_id': 13},
 {'movie_id': 11},
 {'movie_id': 15},
 {'movie_id': 4},
 {'movie_id': 2},
 {'movie_id': 4},
 {'movie_id': 1},
 {'movie_id': 7},
 {'movie_id': 30},
 {'movie_id': 116},
 {'movie_id': 1},
 {'movie_id': 3},
 {'movie_id': 13},
 {'movie_id': 2},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 116},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 116},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 116},
 {'movie_id': 116},
 {'movie_id': 116},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 116},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 2},
 {'movie_id': 2},
 {'movie_id': 2},
 {'movie_id': 1},
 {'movie_id': 2},
 {'movie_id': 1},
 {'movie_id': 116},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 2},
 {'movie_id': 1},
 {'movie_id': 2},
 {'movie_id': 2},
 {'movie_id': 9},
 {'m

In [19]:
db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute('SELECT * FROM movies')
data = cursor.fetchall()

movie_overviews_orig = [row['overview'] for row in data]
movie_id = [row['movie_id'] for row in data]
movie_overviews = [row['overview'] for row in data]
movie_titles = [row['names'] for row in data] 


# Preprocess the movie overviews
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

for n, name in enumerate(movie_overviews):
    temp = name.lower().split(" ")
    temp = [''.join([letter for letter in word if letter.isalnum()]) for word in temp]
    temp = [word for word in temp if word not in stop_words]
    temp = ' '.join(temp)
    movie_overviews[n] = temp

# Calculate cosine similarity
    from sklearn.feature_extraction.text import CountVectorizer

    vectorizer = CountVectorizer().fit_transform(movie_overviews)
    cosine_sim = cosine_similarity(vectorizer)

# Function to get movie recommendations
def get_movie_recommendations( movie_title, similarity_matrix):
    if type(movie_title) == str:
        movie_idx = movie_titles.index(movie_title)
        
    else: 
        movie_idx = movie_title
        print(movie_title)
    similar_movies = list(enumerate(similarity_matrix[movie_idx]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)
    recommended_movies = []
    
    # Calculate a recommendation score based on similarity (cosine similarity)
    for i in similar_movies:
        if(i[1]< 0.15):
            break
        recommended_movie_title = movie_titles[i[0]]
        movie_description = movie_overviews_orig[i[0]]
        recommended_movies.append({'title': recommended_movie_title, 'description': movie_description,'movie_id': movie_id[i[0]], 'score':i[1]})

    return recommended_movies

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kimbe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
db.ping(reconnect=True) 
with db.cursor() as cursor:
    cursor.execute('SELECT * FROM watchedMovies')
    data = cursor.fetchall()[::-1]
    movie_ids = [row['movie_id'] for row in data]
    movie_ids = np.unique(movie_ids)
    # print(data[0]['movie_id'],"ddata")
    temp=[]
    results=[]
    for i in range(len(movie_ids)):
            recommendations = get_movie_recommendations(i, cosine_sim)[1:]
            if len(recommendations) < 1: continue
            temp.append(recommendations)
            if len(temp) > 5: break
        #  print(data[i]['movie_id'])
    
    # results =[row['movie_id'] for row in temp]
    print('-----',temp,'---------------')
    
    for movie_group in temp:
        for movie in movie_group:
            # Access movie information
            
            movie_id = movie['movie_id']
            results.append(movie_id)
    results = np.unique(results)
    print(results)

0


TypeError: 'int' object is not subscriptable